<a href="https://colab.research.google.com/github/HariharaSubramanianM/Loan-Default/blob/master/Final_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports and Normalize

In [ ]:
import pandas as pd

pdict =  pd.read_excel(r"PositiveFactors.xlsx",sheet_name=['Factor1','Factor2','Factor3','Factor4','Factor5','Factor6','Factor7','Factor9','Factor10'])
ndict =  pd.read_excel(r"NegativeFactors.xlsx",sheet_name=['FactorA','FactorB','FactorC','FactorD','FactorE','FactorF','FactorG','FactorH','FactorI'])
clgranks =  pd.read_excel(r"clgranks.xlsx")
clgranks = clgranks.set_index('Name')


In [ ]:

#normalize

def normalize(d1):
  dmax=max(d1.values())[0]
  for key in d1:
    d1[key]=((d1[key][0]/dmax)*10)
  return d1


In [ ]:
#Range Dict
class RangeDict(dict):
    def __getitem__(self, item):
        if not isinstance(item, range): 
            for key in self:
                if item in key:
                    return self[key]
            raise KeyError(item)
        else:
            return super().__getitem__(item)




In [ ]:
def rangegen(d):
  ret=RangeDict({})
  for key,value in d.items():
    start,end = [int(i) for i in key.split('-')]
    dtemp = RangeDict({range(start,end):value})     # left value inclusive, right value exclusive
    ret.update(dtemp)
  return ret


# Find Positive Score

In [ ]:
p1 = normalize(pdict.get('Factor1').set_index('F1').T.to_dict('list'))
p2 = normalize(pdict.get('Factor2').set_index('F2').T.to_dict('list'))
p3 = normalize(pdict.get('Factor3').set_index('F3').T.to_dict('list'))
# p5 = normalize(pdict.get('Factor5').set_index('F5').T.to_dict('list'))
p6 = normalize(pdict.get('Factor6').set_index('F6').T.to_dict('list'))
p9 = normalize(pdict.get('Factor9').set_index('F9').T.to_dict('list'))
p10 =normalize(pdict.get('Factor10').set_index('F10').T.to_dict('list'))

# Range Dicts
p4 = rangegen(normalize(pdict.get('Factor4').set_index('F4').T.to_dict('list')))
p7 = rangegen(normalize(pdict.get('Factor7').set_index('F7').T.to_dict('list')))

In [ ]:
clg='THIAGARAJAR COLLEGE OF ENGINEERING'
p1[clgranks.loc[clg.strip().upper()].NAAC]


9.0

# Find Negative Score

import json
import urllib
import requests

url = 'https://parseapi.back4app.com/classes/india_cities_database?limit=3369&order=ascii_name&keys=ascii_name,population'
headers = {
    'X-Parse-Application-Id': 'k9wzhXDJckHtuuKmMEm8hceXoCZMvxN4CTijZOjn', # This is the fake app's application id
    'X-Parse-Master-Key': 'EpjKI6bUlgkIbOAAUaDKDFj6x2C3AnsZZJqoa4ak' # This is the fake app's readonly master key
}
data = json.loads(requests.get(url, headers=headers).content.decode('utf-8')) # Here you have the data that you need

city=[]
population=[]

for i in data["results"]:
  city.append(i["ascii_name"])
  population.append(i["population"])

cityapi=df = pd.DataFrame(list(zip(city, population)),columns =['City', 'Population'])
filename='CityAPI.xlsx'
cityapi.to_excel(filename)


In [ ]:
# Negative Factor - A
def city_range_gen(d):
  ret=RangeDict({})

  # ref : https://www.rbi.org.in/scripts/bs_viewcontent.aspx?Id=2035#:~:text=Rural%3A%20population%20less%20than%2010%2C000,Metropolitan%3A%2010%20lakh%20and%20above

  city_pop_range={'Rural':[0,10000],'Semi Urban':[10000,100000],'Urban':[100000,1000000],'Metropolitan':[1000000,2000000000]} #2000000000 is rounded max int range
  for key,value in d.items():
    start,end = city_pop_range[key]
    dtemp = RangeDict({range(start,end):value})
    ret.update(dtemp)
  return ret

In [ ]:
# Negative Factor - C
def bor_mar(totamt,getamt,perc):
  diff = totamt-getamt
  rel_diff = (diff/totamt)*100
  risk_val = 10 - rel_diff*(perc/100)
  return risk_val

In [ ]:
nb = normalize(ndict.get('FactorB').set_index('FB').T.to_dict('list'))                   #Marital Status
nh = normalize(ndict.get('FactorH').set_index('FH').T.to_dict('list'))                   #Aadhar with KYV

#RangeDict
na = city_range_gen(normalize(ndict.get('FactorA').set_index('FA').T.to_dict('list')))   #city_range_gen(normalize( {'Metropolitan': [7], 'Rural': [8], 'Semi Urban': [5], 'Urban': [6]} ))
nf = rangegen(normalize(ndict.get('FactorF').set_index('FF').T.to_dict('list')))         #Loan Amount
ng = rangegen(normalize(ndict.get('FactorG').set_index('FG').T.to_dict('list')))         #Repayment Period
ni = rangegen(normalize(ndict.get('FactorI').set_index('FI').T.to_dict('list')))         #Age


#Custom Number
nc = ndict.get('FactorC').set_index('FC').T.to_dict('list')['Margin'][0]                  #Borrower Margin
nd = ndict.get('FactorD').set_index('FD').T.to_dict('list')['SecurityRatio'][0]           #Security or Not
ne = ndict.get('FactorE').set_index('FE').T.to_dict('list')['MaletoFemale'][0]            #gender

In [ ]:
securitydict={"Yes":10/nd, "No":10}
genderdict={"Male":10/ne,"Female":10}

# Calculate Final Score

FINAL SCORE CALCULATION

min positive = 5.98
min negative = 6.0 

max pos-neg = 10-6 = 4
min pos-neg = -4.02

range = 8.02

final value = [ (val+4.02)/8.02 ] * 10

# Positive Score

In [ ]:
def find_pos_score(college,current_course,upskill_course,marks10,marks12,medium,parent_edu,board):
    W1,W2,W3,W4A,W4B,W6,W7,W9,W10 = 4.5,5,4.5,4,4,4,4.5,3.5,4
    S=38
    w1,w2,w3,w4a,w4b,w6,w7,w9,w10 = W1/S,W2/S,W3/S,W4A/S,W4B/S,W6/S,W7/S,W9/S,W10/S

    f1=p1[clgranks.loc[college.strip().upper()].NAAC]
    f2=p2[current_course.strip()]
    f3=p3[upskill_course.strip()]
    f4a = p4[marks10]
    f4b = p4[marks12]
    # f5 = p5(city.strip().upper())
    f6 = p6[medium.strip()]
    f7 = p7[clgranks.loc[college.strip().upper()].Rank]
    f9 = p9[parent_edu.strip()]
    f10 = p10[board.strip()]

    score = w1*f1 + w2*f2 + w3*f3 + w4a*f4a + w4b*f4b + w6*f6 + w7*f7 + w9*f9 + w10*f10
    score = round(score,2)
    return score

In [ ]:
pos_score = find_pos_score("Thiagarajar College of Engineering","BE in CS","IT",92,91,"English","Graduate","State")
print(pos_score)

9.58


# Risk Score

In [ ]:
def find_neg_score(region,mar_status,bow_margin,security,gender,amount,period,kyc,age):
    fA = na[find_pop(region)]
    fB = nb[mar_status]
    fC = bor_mar(bow_margin,amount,nc)
    fD = securitydict[security]
    fE = genderdict[gender]
    fF = nf[amount]
    fG = ng[period]
    fH = nh[kyc]
    fI = ni[age]
    W1,W2,W3,W4,W5,W6,W7,W8,W9 = 3.5,3.5,5,5,4,4.5,4,5,4
    S=38.5
    w1,w2,w3,w4,w5,w6,w7,w8,w9 = W1/S,W2/S,W3/S,W4/S,W5/S,W6/S,W7/S,W8/S,W9/S
    score = w1*fA + w2*fB + w3*fC + w4*fD + w5*fE + w6*fF + w7*fG + w8*fH + w9*fI
    score = round(score,2)
    return score

In [ ]:
neg_score = find_neg_score("Chennai","No",1000000,"Yes","Male",800000,10,"Yes",21)
print(neg_score)

7.31


# Final Score

In [ ]:
final_score = ((pos_score-neg_score+4.02)/8.02)*10
print(final_score)

7.8428927680798015
